In [388]:
import pandas as pd
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


r = 0.01  # Assumed risk-free rate of 1%
# Black-Scholes function for call options
def black_scholes_call(S, K, T, r, sigma):
    """Calculate the Black-Scholes price for a call option."""
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price, d1


# Calculate the ATM call prices and deltas for the next day at 12:00 PM
def calculate_option_info(close_price, strike, sigma, current_time):
    if pd.isnull(sigma):
        return np.nan, np.nan  # Return NaN if sigma is not available
    
    days_to_monday = (7 - current_time.weekday())  
    expiration_time = (current_time + pd.Timedelta(days=days_to_monday)).replace(hour=0, minute=0, second=0, microsecond=0)



    T = (expiration_time - current_time).total_seconds() / (24 * 3600)  # Time in years
    call_price, d1 = black_scholes_call(close_price, strike, T, r, sigma)
    delta = norm.cdf(d1)
    return call_price, delta

In [370]:
data = pd.read_csv('ether_intraday_prices.csv')
data.head()

,date,Open,High,Low,Close,Volume
0,2017-08-17 04:00:00,301.13,301.13,298.00,298.00,5.80167
1,2017-08-17 04:15:00,298.00,300.80,298.00,299.39,31.44065
2,2017-08-17 04:30:00,299.39,300.79,299.39,299.60,52.93579
3,2017-08-17 04:45:00,299.60,302.57,299.60,301.61,35.49066
4,2017-08-17 05:00:00,301.61,302.57,300.95,302.01,81.69235


In [ ]:
df = data
df['date'] = pd.to_datetime(df['date'])

# Function to get the last Monday's close price
def get_last_monday_close(row):
    last_monday = row['date'] + pd.Timedelta(days=(-row['date'].weekday()))  # Last Monday
    last_monday = last_monday.replace(hour=0, minute=0, second=0, microsecond=0)
    last_monday_close = data.loc[data['date'] == last_monday, 'Close']
    return last_monday_close.values[0] if not last_monday_close.empty else None


df['Strike'] = df.apply(get_last_monday_close, axis=1)

# Calculate daily returns
df['returns'] = df['Close'].pct_change()

# Calculate moving standard deviation
window_size = 15 * 24 
df['moving_std'] = df['returns'].rolling(window=window_size).std()
df['sigma'] = df['moving_std'] * np.sqrt(365*15)  

option_info = df.apply(lambda row: calculate_option_info(row['Close'], row['Strike'], row['sigma'], row['date']), axis=1)
df[['7day_call', 'delta']] = pd.DataFrame(option_info.tolist(), index=df.index)
df.to_csv('options.csv')

In [399]:
df

,Unnamed: 0,date,Open,High,Low,Close,Volume,Strike,returns,moving_std,sigma,7day_call,delta
368,368,2017-08-21 00:00:00,299.10,300.52,295.76,299.02,148.37986,299.02,-0.000267,0.007670,0.567545,168.348744,0.787393
369,369,2017-08-21 00:15:00,299.02,300.80,296.13,300.80,95.99531,299.02,0.005953,0.007676,0.567984,169.748038,0.788525
370,370,2017-08-21 00:30:00,300.80,300.90,297.80,300.90,37.82619,299.02,0.000332,0.007676,0.567950,169.714375,0.788405
371,371,2017-08-21 00:45:00,299.70,300.70,299.25,300.70,88.46548,299.02,-0.000665,0.007674,0.567829,169.424384,0.788062
372,372,2017-08-21 01:00:00,300.70,300.70,297.71,298.30,97.18496,299.02,-0.007981,0.007684,0.568556,167.597074,0.786599
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185178,185178,2022-12-03 22:30:00,1257.68,1257.69,1237.00,1240.09,25117.80340,1195.55,-0.013994,0.002325,0.172027,117.184439,0.638652
185179,185179,2022-12-03 22:45:00,1240.08,1243.16,1236.26,1242.01,5400.50180,1195.55,0.001548,0.002321,0.171715,117.790818,0.642167
185180,185180,2022-12-03 23:00:00,1242.02,1243.10,1238.75,1239.67,5328.49570,1195.55,-0.001884,0.002322,0.171835,115.873268,0.638225
185181,185181,2022-12-03 23:15:00,1239.66,1241.48,1238.04,1240.75,4111.04060,1195.55,0.000871,0.002318,0.171532,115.942014,0.640329


In [420]:
df = pd.read_csv('options.csv').dropna()

df['date'] = pd.to_datetime(df['date']) 
df.set_index('date', inplace=True)

initial_investment = 10000  
option_position = 1

# Initialize lists to store results
cash_position = initial_investment
portfolio_values = [cash_position]
daily_deltas = [0]
allocation_proportions = [0]

prev_delta = 0
transaction_cost = 0.003 # Transaction cost of 0.3%

# Simulate the gamma scalping strategy
for i in range(1, len(df)):
    current_price = df['Close'][i]  
    current_option_price = df['7day_call'][i]
    current_delta = df['delta'][i] * option_position
    total_delta = current_delta - prev_delta 
    
    if df.index[i].weekday() == 0 and df.index[i].time() == pd.Timestamp('00:00').time(): # if midnight monday, we sell a new option
        cash_position += current_option_price * option_position 
        cash_position -= current_option_price * option_position * transaction_cost

    # P.S. We do not explicitly model the payout of the option at settlement
    # because due to our delta hedging it's zero 0 

    cash_position -= total_delta * current_price
    cash_position -= total_delta * current_price * transaction_cost

    portfolio_value = cash_position + (current_delta * current_price) - (current_option_price * option_position)
    portfolio_value = round(portfolio_value,2)
    portfolio_values.append(portfolio_value)

    prev_delta = current_delta
    total_delta = total_delta if i != 1 else 0
    daily_deltas.append(total_delta)

    underlying_value = total_delta * current_price 
    allocation_proportion = 1 - cash_position / portfolio_value if portfolio_value != 0 else 0
    allocation_proportions.append(allocation_proportion)

commulative_returns = pd.Series(portfolio_values)/initial_investment - 1
commulative_returns.index = df.index

# Convert returns to daily format
cummulative_daily_returns = commulative_returns.resample('D').last()

# Calculate average daily delta
daily_deltas_series = pd.Series(daily_deltas, index=df.index)
average_daily_delta = daily_deltas_series.resample('D').mean()

# Calculate average daily allocation proportion
allocation_proportions_series = pd.Series(allocation_proportions, index=df.index)
average_allocation_proportion = allocation_proportions_series.resample('D').mean()

In [421]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 3 rows
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        "Daily Returns from Gamma Scalping Strategy",
        "Average Daily Delta",
        "Average Daily Allocation Proportion"
    ),
)

# Plotting Daily Returns
fig.add_trace(
    go.Scatter(
        x=cummulative_daily_returns.index, 
        y=cummulative_daily_returns, 
        mode='lines', 
        name='Daily Returns (%)',
        line=dict(color='blue')
    ),
    row=1, col=1
)

# Plotting Average Daily Delta
fig.add_trace(
    go.Scatter(
        x=average_daily_delta.index, 
        y=average_daily_delta, 
        mode='lines', 
        name='Average Daily Delta',
        line=dict(color='red')
    ),
    row=2, col=1
)

# Plotting Average Daily Allocation Proportion
fig.add_trace(
    go.Scatter(
        x=average_allocation_proportion.index,
        y=average_allocation_proportion,
        name='Average Daily Allocation Proportion',
        marker=dict(color='black')
    ),
    row=3, col=1
)


# Update layout for the combined figure
fig.update_layout(
    height=900,  # Adjust height for better visibility
    width=1600,
    title_text='Gamma Scalping Strategy Analysis',
    template='plotly'
)

# Show the combined figure
fig.show()

In [419]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# Assume daily_returns is your DataFrame/Series with daily return values

daily_returns = cummulative_daily_returns.pct_change().dropna()
average_daily_return = daily_returns.mean()

confidence_level = 0.99 # 99% confidence level
VaR = np.percentile(daily_returns, (1 - confidence_level) * 100)

# 3. Compute Conditional Value at Risk (CVaR)
cvar = daily_returns[daily_returns <= VaR].mean()

# 4. Test Hypothesis against 20% APY
annual_target_return = 0.2/365
t_statistic, p_value = stats.ttest_1samp(daily_returns, annual_target_return, alternative='greater')

# Print the results
print(f"Average Daily Return: {average_daily_return:.4f}")
print(f"Value at Risk (VaR) at 95% Confidence Level: {VaR:.4f}")
print(f"Conditional Value at Risk (CVaR): {cvar:.4f}")
print(f"Z-Score for Hypothesis Test: {t_statistic:.4f}")
print(f"P-Value for Hypothesis Test: {p_value:.4f}")

# Interpretation of the hypothesis test
if p_value < 0.05:
    print("Reject the null hypothesis: the strategy does achieve at least 20% APY on average.")
else:
    print("Fail to reject the null hypothesis: the strategy does not achieve at least 20% APY on average.")

Average Daily Return: 0.0013
Value at Risk (VaR) at 95% Confidence Level: -0.0023
Conditional Value at Risk (CVaR): -0.0058
Z-Score for Hypothesis Test: 12.8520
P-Value for Hypothesis Test: 0.0000
Reject the null hypothesis: the strategy does achieve at least 20% APY on average.
